In [2]:
import pyanitools as pyt
import pyaniasetools as aat
import pyNeuroChem as pyc
import hdnntools as hdt
import numpy as np

from ase_interface import ANIENS
from ase_interface import ensemblemolecule
from ase.atoms import Atoms

import matplotlib.pyplot as plt
import aniensemblestats as ens

/scratch/Anaconda/anaconda3/envs/my-rdkit-env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


van Der Waals correction will be unavailable. Please install ased3


In [3]:
#h5files = '/home/jujuman/Research/DataReductionMethods/modelCNOSFCl/initial/cv1/testset/testset1.h5'
#h5files = '/home/jujuman/Seagate2TB/Research/ANI-DATASET/h5data/ani_data_c10test.h5'
#h5files = '/home/jujuman/Research/extensibility_test_sets/gdb-10/gdb11_10_test500.h5'
h5files = '/nh/nest/u/jsmith/Research/gutzwiller_research/new_converged_data/20180723-MD/N100/U10.0-N100.h5'

#wkdir = '/nh/nest/u/jsmith/scratch/Research/levi_test/ANI-2x_v62518'
wkdir = '/nh/nest/u/jsmith/Research/gutzwiller_research/new_converged_data/20180723-MD/model-gutz-U10.000-N33_new-x8'

cns = wkdir + '/train0/rX-2.95R_32-2.2A_a8-8.params'
sae = wkdir + '/train0/sae.dat'
nnf = wkdir + '/train'
Nn = 1

In [4]:
# Set NC
aens = ensemblemolecule(cns, sae, nnf, Nn, 0)

In [5]:
#anicv = aat.anicrossvalidationconformer(cns, sae, nnf, Nn, [0])
#nc = pyc.molecule(cns,sae,nnf+str(0)+'/networks/', 0)

L=8.682536
cell = np.array([[L, 0, 0],
                 [0, L, 0],
                 [0, 0, L]],dtype=np.float32)

celi = (np.linalg.inv(cell)).astype(np.float32)

aens.set_pbc(True,True,True)
aens.set_cell((cell).astype(np.float32), celi)

In [6]:
adl = pyt.anidataloader(h5files)

In [7]:
hfile = '/nh/nest/u/jsmith/Research/gutzwiller_research/new_converged_data/20180723-MD/N100/U10.0-N10-ANI.h5'
dpack = pyt.datapacker(hfile)

In [8]:
Eani_t = []
Edft_t = []
dEani_t = []
dEdft_t = []
Fani_t = []
Fdft_t = []
for i,data in enumerate(adl):
    X = data['coordinates']
    S = data['species']
    Edft = data['energies']
    Fdft = data['forces']
    

    #Fdft = data['forces']

    #if 'N' in S:
    Eani = []
    Fani = []
    for x in X:
        aens.set_molecule(x.astype(np.float32), S)
        energy, force, stddev, Fstddev = aens.compute_mean_props()
        Eani.append(energy)
        Fani.append(force.reshape(1,-1,3))

    Eani = np.array(Eani)
    Fani = np.vstack(Fani)
    
    print(Fani.shape)
    #Eani, Fani, Sig = anicv.compute_energyandforce_conformations(X=np.array(X,dtype=np.float32),S=S)
    #print(Fdft)
    #Eani = np.asarray(np.mean(Eani,axis=1))
    #print(Eani.shape,Edft.shape)
    #idx = np.where((Edft - Edft.min() < 300.0))
    #print(np.abs(Edft-Eani), len(S))
    
    #mina = Eani.min()
    #mind = Edft.min()
    
    #Esae1 = hdt.hatokcal*hdt.compute_sae(sae,S)
    #Esae2 = hdt.hatokcal*hdt.compute_sae('/nh/nest/u/jsmith/scratch/Research/levi_test/train/sae_linfit.dat',S)
    
    Eani_t.append(Eani-Eani.min())
    Edft_t.append(Edft-Edft.min())
    
    Fani_t.append(Fani)
    Fdft_t.append(Fdft)
    
    #dEani_t.append(hdt.calculatedmat(Eani))
    #dEdft_t.append(hdt.calculatedmat(Edft))
    #print(np.abs(-Fani[0]-hdt.hatokcal*Fdft))

Eani_t = np.concatenate(Eani_t)
Edft_t = np.concatenate(Edft_t)
Fani_t = np.vstack(Fani_t)
Fdft_t = np.vstack(Fdft_t)
#dEani_t = np.concatenate(dEani_t)
#dEdft_t = np.concatenate(dEdft_t)

print(Fani_t.shape)

new_data = dict()

new_data["species"] = S
new_data["coordinates"] = X
new_data["energies"] = Eani_t
new_data["forces"] = Fani_t

dpack.store_data('/kt015/ANI',**new_data)
dpack.cleanup()

/scratch/Gits/NeuroChem/build/lib/ase_interface.py:516: RuntimeWarning: invalid value encountered in true_divide
  v2 = np.sum(np.sum(np.power(dF, 2), axis=0) / (self.Nn * (self.Nn - 1)))


(510, 100, 3)
(510, 100, 3)


In [ ]:
print(,Fani_t)
#Eani_t = np.concatenate(Eani_t)
#Edft_t = np.concatenate(Edft_t)
#print(Eani_t.shape, Edft_t.shape)

print('ERMS:',hdt.calculaterootmeansqrerror(Eani_t,Edft_t))
print('EMAE:',hdt.calculatemeanabserror(Eani_t,Edft_t))
print('FRMS:',hdt.calculaterootmeansqrerror(Fani_t,Fdft_t))
print('FMAE:',hdt.calculatemeanabserror(Fani_t,Fdft_t))#
#print(hdt.calculaterootmeansqrerror(dEani_t,dEdft_t))
#print(hdt.calculatemeanabserror(dEani_t,dEdft_t))
#print(Eani_t.size)

In [ ]:
ens.plot_corr_dist(Fdft_t,Fani_t,xlabel='Target Force',ylabel='Predicted Force')

In [ ]:
ens.plot_corr_dist(Edft_t,Eani_t,xlabel='Target Energy',ylabel='Predicted Energy', inset=False)

In [ ]:
fig = plt.figure(figsize=[10,10])
plt.hist(Fdft_t,bins=100)
plt.yscale("log")
plt.show()

In [ ]:
fig = plt.figure(figsize=[10,10])
plt.hist(np.linalg.norm(Fdft_t,axis=1),bins=100)
plt.yscale("log")
plt.xlabel("Gutz force magnitude")
plt.ylabel("Count")
plt.show()